In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torchvision

import matplotlib.pyplot as plt
import lightning as L

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from utils.FFT import spectrogram, stft

/nfs/ksdata/tran/miniconda3/envs/python_3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

proxy = 'http://proxy-k.ecl.ntt.co.jp:8080'

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [3]:
# Concat data
save_folder = os.path.join("dataset", "processed_concat_data")
os.makedirs(save_folder, exist_ok=True)

In [4]:
TEST = False

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
L.seed_everything(random_seed)

Seed set to 1


1

## FFT Data

In [5]:
data_key = ["LAcc_x", "LAcc_y", "LAcc_z", "Gyr_x", "Gyr_y", "Gyr_z"]

data_path_dict = dict(zip(data_key, 
    [os.path.join("dataset", "Torso", x + ".txt") for x in data_key]))

validate_path_dict = dict(zip(data_key, 
    [os.path.join("dataset", "validate", "Torso", x + ".txt") for x in data_key]))

In [6]:
data_dict = {}

for data_name in data_key:
    train_data = np.loadtxt(data_path_dict[data_name])
    val_data = np.loadtxt(validate_path_dict[data_name])
    
    one_data = np.concatenate([train_data, val_data], axis=0)
    data_dict[data_name] = stft(one_data, verbose=True)[1]

data_dict.keys()


data1d.shape (104124500,)
freqs.shape (257,)
specs.shape (257, 208249)
data1d.shape (104124500,)
freqs.shape (257,)
specs.shape (257, 208249)
data1d.shape (104124500,)
freqs.shape (257,)
specs.shape (257, 208249)
data1d.shape (104124500,)
freqs.shape (257,)
specs.shape (257, 208249)
data1d.shape (104124500,)
freqs.shape (257,)
specs.shape (257, 208249)
data1d.shape (104124500,)
freqs.shape (257,)
specs.shape (257, 208249)


dict_keys(['LAcc_x', 'LAcc_y', 'LAcc_z', 'Gyr_x', 'Gyr_y', 'Gyr_z'])

In [7]:
stack = False

In [8]:
## STACK

if stack:
    STACK_FREQ_NUMBER = 8

    one_data = data_dict[data_name]
    data_array_list = []

    for freq_step in range(STACK_FREQ_NUMBER):
        start_idx = freq_step
        end_idx = - STACK_FREQ_NUMBER + freq_step + 1
        if end_idx == 0:
            end_idx = None

        data_array = one_data[:, start_idx: end_idx]
        print("data_array.shape", data_array.shape)
        data_array_list.append(data_array)

    one_data_stacked = np.stack(data_array_list)
    print("one_data_stacked.shape", one_data_stacked.shape)

    data_stacked_list = []

    for data_name in data_key:
        one_data = data_dict[data_name]
        data_array_list = []

        for freq_step in range(STACK_FREQ_NUMBER):
            start_idx = freq_step
            end_idx = - STACK_FREQ_NUMBER + freq_step + 1
            if end_idx == 0:
                end_idx = None

            data_array = one_data[:, start_idx: end_idx]
            data_array_list.append(data_array)

        one_data_stacked = np.stack(data_array_list)
        data_stacked_list.append(one_data_stacked)

    data_stacked = np.stack(data_stacked_list)
    print("data_stacked.shape", data_stacked.shape)

    data_stacked = data_stacked.transpose(3, 0, 1, 2)
    print("data_stacked.shape", data_stacked.shape)

    np.save(os.path.join(save_folder, "torso_fft.npy") ,data_stacked)
    

In [9]:
## NO STACK
if not stack:
    data_stacked_list = []

    for data_name in data_key:
        print(data_dict[data_name].shape)
        data_stacked_list.append(data_dict[data_name])
        
    data_stacked = np.stack(data_stacked_list)
    print("data_stacked.shape", data_stacked.shape)

    data_stacked = data_stacked.transpose(2, 0, 1)
    print("data_stacked.shape", data_stacked.shape)
    
    np.save(os.path.join(save_folder, "torso_fft.npy") ,data_stacked)
    

(257, 208249)
(257, 208249)
(257, 208249)
(257, 208249)
(257, 208249)
(257, 208249)
data_stacked.shape (6, 257, 208249)
data_stacked.shape (208249, 6, 257)


## Label

In [10]:
label_train_path = os.path.join("dataset", "Torso", "Label.txt")
label_val_path = os.path.join("dataset", "validate", "Torso", "Label.txt")

label_train = np.loadtxt(label_train_path)
label_val = np.loadtxt(label_val_path)

label = np.concatenate([label_train, label_val], axis=0)
label.shape

(208249, 500)

In [11]:
if stack:
    data_label = label[: - STACK_FREQ_NUMBER + 1, :]
    print(data_label.shape)
    np.save(os.path.join(save_folder, "torso_label.npy"), data_label)

In [12]:
if not stack:
    np.save(os.path.join(save_folder, "torso_label.npy"), label)